
Feedback : shins777@gmail.com

이 Colab 은 대부분의 고객의 요구사항이면서, 많은 유즈케이스에서의 근간이 되는 지식검색에 대한 예제입니다.
Gronding service 로 Vertex AI Search와 Langchain 을 연동해서 정보를 검색하고 해당 검색된 정보를 Gemini 를 통해서 추론해서 답해주는 형태의 예제입니다.

지식 검색의 대상은 "정보통신 진흥 및 융합 활성화 등에 관한 특별법" 이며 해당 특별법은 아래 사이트에서 다운받았습니다.
https://www.law.go.kr/

#라이브러리 설치
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [3]:
!pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai google-cloud-discoveryengine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.6 MB/s eta 0:00:00


#GCP 사용자 인증

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
DATA_STORE_ID = "hackathon-ds_1709838247983"
MODEL = "gemini-pro"

# Vertex AI Search 검색
아래 해당 문서에 대해서 Vector 검색을 위해서는 Vertex AI Search 내에서 해당 검색 처리에 대한 구성을 해야 합니다.
해당 솔루션의 메뉴얼은 아래와 같습니다.

https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search

해당 Vertex AI 구성이 완료되면 아래 Langchain 함수를 통해서 처리를 구성할 수 있습니다.

*   https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search



In [6]:
from langchain.retrievers import (
    GoogleVertexAIMultiTurnSearchRetriever,
    GoogleVertexAISearchRetriever,
)

DOC_REGION="global"

retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=DOC_REGION,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    # max_extractive_answer_count=2,
    # get_extractive_answers=True,
    max_extractive_segment_count=1,
    get_extractive_segments=True,
    query_expansion_condition=2,
    engine_data_type = 0
)


In [17]:
query = "특별법에서 민간 부문의 양자정보통신기술개발 지원에 대해서 상세하게 설명해주세요."

result = retriever.get_relevant_documents(query)

context ={}

for doc in result:
    print(doc.json)
    context[doc.metadata['source']] = doc.page_content
    print("\n\n======================")
    print(doc.metadata['source'])
    print(doc.page_content)

print(context)


<bound method BaseModel.json of Document(page_content='제27조의3(민간 부문의 양자정보통신기술개발 지원) ① 정부는 민간 부문의 양자정보통신기술에 대한 연구개발투자가\n효율적으로 추진될 수 있도록 행정적ㆍ기술적 지원 등 필요한 지원을 할 수 있다.\n② 정부는 양자정보통신기술을 이용한 기술집약형 중소기업 및 벤처기업과 양자정보통신기술을 이용하여 창업하\n는 기업에 대하여 제1항에 따른 지원시책을 우선적으로 추진할 수 있다.\n[본조신설 2020. 6. 9.]\n제27조의3 삭제 <2023. 10. 31.>\n[시행일: 2024. 11. 1.] 제27조의3\n제27조의4(양자정보통신산업클러스터의 지정 등) ① 과학기술정보통신부장관은 양자정보통신 신기술의 창출 및 확산,\n인력 양성, 산업 경쟁력 강화 등을 촉진하기 위해 기업, 연구소, 대학 등을 상호 연계하여 집중적으로 조성하기 위한\n지역(이하 “양자정보통신산업클러스터”라 한다)을 관할 시ㆍ도지사의 의견을 듣고 관계 중앙행정기관의 장과 협의한\n후 전략위원회의 심의를 거쳐 지정할 수 있다.\n② 과학기술정보통신부장관은 제1항에 따라 양자정보통신산업클러스터를 지정한 경우에는 대통령령으로 정하는\n바에 따라 그 내용을 관보에 고시하고, 지체 없이 이를 관할 시ㆍ도지사에게 통보하여야 한다. 이 경우 지형도면의\n고시에 관하여는 「토지이용규제 기본법」 제8조에 따른다.\n③ 제2항에 따른 통보를 받은 관할 시ㆍ도지사는 그 내용을 14일 이상 주민이 열람할 수 있도록 하여야 한다.\n④ 그 밖에 양자정보통신산업클러스터의 지정 등에 필요한 사항은 대통령령으로 정한다.\n[본조신설 2020. 6. 9.]\n제27조의4 삭제 <2023. 10. 31.>\n[시행일: 2024. 11. 1.] 제27조의4\n제28조(공공부문의 정보통신장비 구축사업) ① 중앙행정기관, 지방자치단체 및 공공기관의 장은 정보통신장비 구축사\n업의 계약을 체결하는 경우 「국가를 당사자로 하는 계약에 관한 

# Gemini Pro 실행 - Vertex AI Search as a Grounding Service


Responsible AI setting
*   HarmCategory : https://cloud.google.com/vertex-ai/docs/reference/rest/v1/HarmCategory
*   HarmBlockThreshold : https://cloud.google.com/php/docs/reference/cloud-ai-platform/0.31.0/V1.SafetySetting.HarmBlockThreshold



In [18]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
                    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

In [19]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  safety_settings = safety_settings,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

In [21]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 쉽게 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=query)

print(f"Prompt : {prompt}")
print(f"Answer : {gemini_pro.invoke(prompt)}")


Prompt : 

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 쉽게 답변해주세요.
  Context : {'gs://hackathon-kr-2024/정보통신 진흥 및 융합 활성화 등에 관한 특별법(법률)(제19240호)(20230622).pdf': '제27조의3(민간 부문의 양자정보통신기술개발 지원) ① 정부는 민간 부문의 양자정보통신기술에 대한 연구개발투자가\n효율적으로 추진될 수 있도록 행정적ㆍ기술적 지원 등 필요한 지원을 할 수 있다.\n② 정부는 양자정보통신기술을 이용한 기술집약형 중소기업 및 벤처기업과 양자정보통신기술을 이용하여 창업하\n는 기업에 대하여 제1항에 따른 지원시책을 우선적으로 추진할 수 있다.\n[본조신설 2020. 6. 9.]\n제27조의3 삭제 <2023. 10. 31.>\n[시행일: 2024. 11. 1.] 제27조의3\n제27조의4(양자정보통신산업클러스터의 지정 등) ① 과학기술정보통신부장관은 양자정보통신 신기술의 창출 및 확산,\n인력 양성, 산업 경쟁력 강화 등을 촉진하기 위해 기업, 연구소, 대학 등을 상호 연계하여 집중적으로 조성하기 위한\n지역(이하 “양자정보통신산업클러스터”라 한다)을 관할 시ㆍ도지사의 의견을 듣고 관계 중앙행정기관의 장과 협의한\n후 전략위원회의 심의를 거쳐 지정할 수 있다.\n② 과학기술정보통신부장관은 제1항에 따라 양자정보통신산업클러스터를 지정한 경우에는 대통령령으로 정하는\n바에 따라 그 내용을 관보에 고시하고, 지체 없이 이를 관할 시ㆍ도지사에게 통보하여야 한다. 이 경우 지형도면의\n고시에 관하여는 「토지이용규제 기본법」 제8조에 따른다.\n③ 제2항에 따른 통보를 받은 관할 시ㆍ도지사는 그 내용을 14일 이상 주민이 열람할 수 있도록 하여야 한다.\n④ 그 밖에 양자정보통신산업클러스터의 지정